In [2]:
import pandas as pd
import numpy as np

In [2]:
balance_data = pd.read_csv('a_balance_data.csv')
# Group by stock code and calculate various statistical aggregates for financial metrics
features_df = balance_data.groupby('ts_code').agg({
    'total_cur_assets': ['sum', 'mean', 'std'],  # Current assets aggregates
    'total_assets': ['sum', 'mean', 'std'],  # Total assets aggregates
    'fix_assets': ['sum', 'mean', 'std'],  # Fixed assets aggregates
    'intan_assets': ['sum', 'mean', 'std'],  # Intangible assets aggregates
    'r_and_d': ['sum', 'mean', 'std'],  # R&D expenses aggregates
    'total_nca': ['sum', 'mean', 'std'],  # Non-current assets aggregates
    'total_liab': ['sum', 'mean', 'std'],  # Total liabilities aggregates
    'total_cur_liab': ['sum', 'mean', 'std'],  # Current liabilities aggregates
    'long_pay_total': ['sum', 'mean', 'std'],  # Long-term payables aggregates
    'total_hldr_eqy_exc_min_int': ['sum', 'mean', 'std'],  # Shareholder equity excluding minority interests aggregates
    'total_liab_hldr_eqy': ['sum', 'mean', 'std'],  # Liabilities and shareholder equity aggregates
    'lt_payable': ['sum', 'mean', 'std'],  # Long-term payables aggregates
    'oth_pay_total': ['sum', 'mean', 'std']  # Other payables aggregates
}).reset_index()

# Flatten the MultiIndex in columns created by groupby.agg() operations
features_df.columns = ['ts_code'] + [f'{col[0]}_{col[1]}' for col in features_df.columns.values[1:]]

In [3]:
# Basic Financial Ratios
features_df['asset_to_liability_ratio'] = features_df['total_assets_sum'] / features_df['total_liab_sum']
features_df['current_ratio'] = features_df['total_cur_assets_sum'] / features_df['total_cur_liab_sum']
features_df['quick_ratio'] = features_df['total_cur_assets_sum'] / features_df['total_cur_liab_sum']
features_df['debt_to_equity_ratio'] = features_df['total_liab_sum'] / features_df['total_hldr_eqy_exc_min_int_sum']
features_df['equity_to_asset_ratio'] = features_df['total_hldr_eqy_exc_min_int_sum'] / features_df['total_assets_sum']

# Additional Financial Metrics
features_df['working_capital'] = features_df['total_cur_assets_sum'] - features_df['total_cur_liab_sum']
features_df['working_capital_ratio'] = features_df['working_capital'] / features_df['total_assets_sum']
features_df['liability_to_asset_ratio'] = features_df['total_liab_sum'] / features_df['total_assets_sum']
features_df['fixed_asset_turnover_ratio'] = features_df['total_assets_sum'] / features_df['fix_assets_sum']
features_df['intangible_to_total_asset_ratio'] = features_df['intan_assets_sum'] / features_df['total_assets_sum']
features_df['research_to_asset_ratio'] = features_df['r_and_d_sum'] / features_df['total_assets_sum']

In [4]:
print(features_df.head())


     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [5]:
features_df.to_csv('a_stock_features_comprehensive.csv', index=False)


In [6]:
# Load cash flow statement data
cashflow_data = pd.read_csv('a_cashflow_data.csv')


In [7]:
# Group by stock code and compute cash flow-related features
cash_features = cashflow_data.groupby('ts_code').agg({
    'n_cashflow_act': ['sum', 'mean', 'std'],  # Net cash flow from operating activities
    'n_cashflow_inv_act': ['sum', 'mean', 'std'],  # Net cash flow from investing activities
    'n_cash_flows_fnc_act': ['sum', 'mean', 'std'],  # Net cash flow from financing activities
    'free_cashflow': ['sum', 'mean', 'std'],  # Free cash flow
    'c_cash_equ_end_period': ['last'],  # Ending balance of cash and cash equivalents
    'c_cash_equ_beg_period': ['first'],  # Beginning balance of cash and cash equivalents
    'eff_fx_flu_cash': ['sum', 'mean', 'std'],  # Effect of exchange rate changes on cash
    'net_dism_capital_add': ['sum', 'mean'],  # Net increase in capital expenditures
    'net_cash_rece_sec': ['sum', 'mean'],  # Net cash received from securities
    'c_prepay_amt_borr': ['sum', 'mean'],  # Cash paid for prepayment of borrowings
    'c_pay_dist_dpcp_int_exp': ['sum', 'mean']  # Cash paid for dividends, profit distributions, and interest expenses
}).reset_index()

# Flatten the MultiIndex in columns created by groupby.agg() operations
cash_features.columns = ['ts_code'] + [f'{col[0]}_{col[1]}' for col in cash_features.columns.values[1:]]


In [8]:
# Merge cash flow features into the comprehensive features DataFrame
features_df = features_df.merge(cash_features, on='ts_code', how='left')

In [11]:
# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_with_cashflow.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [13]:
# Load daily stock data
daily_data = pd.read_csv('a_daily_data.csv')
# Group by stock code and compute basic features related to price and volume
daily_features = daily_data.groupby('ts_code').agg({
    'close': ['mean', 'std', 'max', 'min'],
    'vol': ['sum', 'mean', 'std'],
    'amount': ['sum', 'mean', 'std'],
    'pct_chg': ['mean', 'std', 'sum']
}).reset_index()

# Flatten the MultiIndex for easier access
daily_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in daily_features.columns.values[1:]
]

# Calculate additional features like volatility and trading momentum
daily_data['log_return'] = np.log(daily_data['close'] / daily_data['close'].shift(1))
volatility_features = daily_data.groupby('ts_code')['log_return'].agg(volatility='std').reset_index()

# Moving averages and exponential moving averages for different windows (e.g., 7, 30, 90 days)
for window in [7, 30, 90]:
    daily_data[f'sma_{window}'] = daily_data.groupby('ts_code')['close'].transform(lambda x: x.rolling(window=window).mean())
    daily_data[f'ema_{window}'] = daily_data.groupby('ts_code')['close'].transform(lambda x: x.ewm(span=window, adjust=False).mean())

# Merge daily data to calculate differences from moving averages
ma_features = daily_data.groupby('ts_code').agg({
    'sma_7': 'last',
    'sma_30': 'last',
    'sma_90': 'last',
    'ema_7': 'last',
    'ema_30': 'last',
    'ema_90': 'last'
}).reset_index()

# Combine all features
daily_features = daily_features.merge(volatility_features, on='ts_code', how='left')
daily_features = daily_features.merge(ma_features, on='ts_code', how='left')


In [14]:
# Merge daily stock performance features into the comprehensive features DataFrame
features_df = features_df.merge(daily_features, on='ts_code', how='left')

In [15]:
# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_with_advanced_daily.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [17]:
# Load dividend data
dividend_data = pd.read_csv('a_dividend_data.csv')

# Group by stock code and compute dividend-related features
dividend_features = dividend_data.groupby('ts_code').agg({
    'cash_div': ['sum', 'mean', 'max', 'count'],  # Total, average, and maximum cash dividend, and count of dividends
    'stk_div': ['sum', 'mean', 'max']  # Total, average, and maximum stock dividend
}).reset_index()

# Flatten the MultiIndex for easier access
dividend_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in dividend_features.columns.values[1:]
]

# Additional dividend growth and frequency features
# Calculate yearly dividend growth rate (requires dividend data sorted by date)
dividend_data['prev_cash_div'] = dividend_data.groupby('ts_code')['cash_div'].shift(1)
dividend_data['div_growth'] = (dividend_data['cash_div'] - dividend_data['prev_cash_div']) / dividend_data['prev_cash_div']
div_growth_features = dividend_data.groupby('ts_code')['div_growth'].agg(['mean', 'std']).reset_index()  # Average and standard deviation of dividend growth

# Combine features
dividend_features = dividend_features.merge(div_growth_features, on='ts_code', how='left')

In [18]:
# Merge dividend features into the comprehensive features DataFrame
features_df = features_df.merge(dividend_features, on='ts_code', how='left')

In [19]:
# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_with_dividends.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [20]:
# Load forecast data
forecast_data = pd.read_csv('a_forecast_data.csv')
# Convert ann_date to datetime to manage time-series data effectively
forecast_data['ann_date'] = pd.to_datetime(forecast_data['ann_date'])
forecast_data.sort_values(by=['ts_code', 'ann_date'], inplace=True)

# Initial aggregation of profit change predictions
forecast_features = forecast_data.groupby('ts_code').agg({
    'p_change_min': ['mean', 'std', 'min', 'max'],
    'p_change_max': ['mean', 'std', 'min', 'max'],
    'net_profit_min': ['mean', 'std', 'min', 'max'],
    'net_profit_max': ['mean', 'std', 'min', 'max']
}).reset_index()

forecast_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in forecast_features.columns.values[1:]
]

# Calculate the changes in net profit forecasts to capture trends
forecast_data['net_profit_change'] = forecast_data.groupby('ts_code')['net_profit_max'].diff()
forecast_data['percent_profit_change'] = forecast_data['net_profit_change'] / forecast_data['net_profit_max'].shift(1)

change_features = forecast_data.groupby('ts_code').agg({
    'net_profit_change': ['mean', 'std', 'sum'],
    'percent_profit_change': ['mean', 'std']
}).reset_index()

change_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in change_features.columns.values[1:]
]

# Merge change features
forecast_features = forecast_features.merge(change_features, on='ts_code', how='left')

# Aggregate forecast update frequencies and types per year
forecast_data['year'] = forecast_data['ann_date'].dt.year
annual_update_count = forecast_data.groupby(['ts_code', 'year']).size().groupby('ts_code').mean().reset_index(name='avg_annual_updates')
unique_types_per_year = forecast_data.groupby(['ts_code', 'year'])['type'].nunique().groupby('ts_code').mean().reset_index(name='avg_unique_types_per_year')

# Merge frequency and diversity metrics
forecast_features = forecast_features.merge(annual_update_count, on='ts_code', how='left')
forecast_features = forecast_features.merge(unique_types_per_year, on='ts_code', how='left')

In [22]:
# Merge the forecast features into the comprehensive features DataFrame
features_df = features_df.merge(forecast_features, on='ts_code', how='left')

# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_final.csv', index=False)


     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [26]:
# Load holder data
holder_data = pd.read_csv('a_holder_data.csv')
# Convert ann_date to datetime for sorting and time-based grouping
holder_data['ann_date'] = pd.to_datetime(holder_data['ann_date'])
holder_data.sort_values(by=['ts_code', 'ann_date'], inplace=True)

# Initial aggregate features
holder_features = holder_data.groupby('ts_code').agg({
    'hold_amount': ['sum', 'mean', 'max'],  # Total, average, and maximum holding amount
    'hold_ratio': ['mean', 'std', 'max'],   # Average, standard deviation, and maximum holding ratio
    'hold_change': ['sum', 'mean', 'std']   # Sum, mean, and standard deviation of holding changes
}).reset_index()

# Flatten the MultiIndex for easier access
holder_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in holder_features.columns.values[1:]
]

# Additional features on holder type distribution
holder_type_counts = holder_data.pivot_table(index='ts_code', columns='holder_type', values='hold_amount', aggfunc='count', fill_value=0).reset_index()
holder_type_counts.columns = ['ts_code'] + [f'holder_count_{col}' for col in holder_type_counts.columns if col != 'ts_code']

# Merge holder type features
holder_features = holder_features.merge(holder_type_counts, on='ts_code', how='left')

# Detect significant holder changes (as a proxy for potential buyouts or large transactions)
holder_data['change_pct'] = holder_data.groupby('ts_code')['hold_amount'].pct_change()
significant_changes = holder_data.groupby('ts_code')['change_pct'].agg(lambda x: (x.abs() > 0.1).sum()).reset_index(name='num_significant_changes')

# Merge significant changes into holder features
holder_features = holder_features.merge(significant_changes, on='ts_code', how='left')

In [27]:
# Merge the shareholder features into the comprehensive features DataFrame
features_df = features_df.merge(holder_features, on='ts_code', how='left')

# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_with_holder_data.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [28]:
# Load income data
income_data = pd.read_csv('a_income_data.csv')
# Convert ann_date and f_ann_date to datetime for potential time-based analysis
income_data['ann_date'] = pd.to_datetime(income_data['ann_date'])
income_data['f_ann_date'] = pd.to_datetime(income_data['f_ann_date'])
income_data.sort_values(by=['ts_code', 'f_ann_date'], inplace=True)

# Initial aggregate features from income statement data
income_features = income_data.groupby('ts_code').agg({
    'total_revenue': ['sum', 'mean', 'std'],  # Sum, average, and standard deviation of total revenue
    'operate_profit': ['sum', 'mean', 'std'],  # Operating profit
    'non_oper_income': ['sum', 'mean', 'std'],  # Non-operating income
    'non_oper_exp': ['sum', 'mean', 'std'],  # Non-operating expenses
    'total_profit': ['sum', 'mean', 'std'],  # Total profit
    'income_tax': ['sum', 'mean', 'std'],  # Income tax
    'n_income': ['sum', 'mean', 'std'],  # Net income
    'ebit': ['sum', 'mean', 'std'],  # Earnings before interest and taxes
    'ebitda': ['sum', 'mean', 'std'],  # Earnings before interest, taxes, depreciation, and amortization
}).reset_index()

# Flatten the MultiIndex for easier access
income_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in income_features.columns.values[1:]
]

# Calculating ratios and additional metrics
income_data['profit_margin'] = income_data['n_income'] / income_data['total_revenue']
income_data['operating_margin'] = income_data['operate_profit'] / income_data['total_revenue']
income_data['tax_rate'] = income_data['income_tax'] / income_data['total_profit']

# Aggregate these ratios
ratio_features = income_data.groupby('ts_code').agg({
    'profit_margin': ['mean', 'std'],
    'operating_margin': ['mean', 'std'],
    'tax_rate': ['mean', 'std']
}).reset_index()

ratio_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in ratio_features.columns.values[1:]
]

# Merge ratio features into the main income feature DataFrame
income_features = income_features.merge(ratio_features, on='ts_code', how='left')

In [29]:
# Merge the income statement features into the comprehensive features DataFrame
features_df = features_df.merge(income_features, on='ts_code', how='left')

# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_with_income_data.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [30]:
# Load valuation data
valuation_data = pd.read_csv('a_valuation_data.csv')

# Convert trade_date to datetime to facilitate time-series analysis
valuation_data['trade_date'] = pd.to_datetime(valuation_data['trade_date'])
valuation_data.sort_values(by=['ts_code', 'trade_date'], inplace=True)

# Initial aggregation of valuation and trading metrics
valuation_features = valuation_data.groupby('ts_code').agg({
    'close': ['mean', 'std', 'max', 'min'],  # Mean, standard deviation, maximum, and minimum closing prices
    'turnover_rate': ['mean', 'std'],  # Mean and standard deviation of turnover rate
    'volume_ratio': ['mean', 'std'],  # Mean and standard deviation of volume ratio
    'pe': ['mean', 'std'],  # Mean and standard deviation of price-to-earnings ratio
    'pe_ttm': ['mean', 'std'],  # Mean and standard deviation of PE trailing twelve months
    'pb': ['mean', 'std'],  # Mean and standard deviation of price-to-book ratio
    'ps': ['mean', 'std'],  # Mean and standard deviation of price-to-sales ratio
    'ps_ttm': ['mean', 'std'],  # Mean and standard deviation of PS trailing twelve months
    'dv_ratio': ['mean', 'std'],  # Mean and standard deviation of dividend yield ratio
    'dv_ttm': ['mean', 'std'],  # Mean and standard deviation of dividend yield trailing twelve months
    'total_mv': ['mean', 'std', 'max', 'min'],  # Market capitalization metrics
    'circ_mv': ['mean', 'std', 'max', 'min']  # Circulating market capitalization metrics
}).reset_index()

# Flatten the MultiIndex for easier access
valuation_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in valuation_features.columns.values[1:]
]

# Additional features on market response to valuation changes
valuation_data['pe_change'] = valuation_data.groupby('ts_code')['pe'].pct_change()
valuation_data['pb_change'] = valuation_data.groupby('ts_code')['pb'].pct_change()

# Aggregate these changes
change_features = valuation_data.groupby('ts_code').agg({
    'pe_change': ['mean', 'std', 'sum'],
    'pb_change': ['mean', 'std', 'sum']
}).reset_index()

change_features.columns = ['ts_code'] + [
    f'{col[0]}_{col[1]}' for col in change_features.columns.values[1:]
]

# Merge change features into the main valuation feature DataFrame
valuation_features = valuation_features.merge(change_features, on='ts_code', how='left')

/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_93869/954447159.py:30: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  valuation_data['pe_change'] = valuation_data.groupby('ts_code')['pe'].pct_change()
/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_93869/954447159.py:31: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  valuation_data['pb_change'] = valuation_data.groupby('ts_code')['pb'].pct_change()


In [31]:
# Merge the valuation features into the comprehensive features DataFrame
features_df = features_df.merge(valuation_features, on='ts_code', how='left')

# Display the first few rows of the DataFrame to view the merged results
print(features_df.head())

# Save the results to a CSV file
features_df.to_csv('a_comprehensive_features_data.csv', index=False)

     ts_code  total_cur_assets_sum  total_cur_assets_mean  \
0  000001.SZ          0.000000e+00                    NaN   
1  000002.SZ          7.416524e+13           7.416524e+11   
2  000004.SZ          2.489588e+10           2.489588e+08   
3  000006.SZ          1.038808e+12           1.038808e+10   
4  000007.SZ          2.704297e+10           2.704297e+08   

   total_cur_assets_std  total_assets_sum  total_assets_mean  \
0                   NaN      2.847095e+14       2.847095e+12   
1          5.391315e+11      8.735653e+13       8.735653e+11   
2          1.635394e+08      4.687988e+10       4.687988e+08   
3          4.860001e+09      1.263923e+12       1.263923e+10   
4          1.196337e+08      5.038727e+10       5.038727e+08   

   total_assets_std  fix_assets_sum  fix_assets_mean  fix_assets_std  ...  \
0      1.713327e+12    6.480370e+11     6.480370e+09    3.741434e+09  ...   
1      6.715046e+11    6.496551e+11     6.496551e+09    5.851365e+09  ...   
2      4.635791e+

In [19]:
df1 = pd.read_csv('company_major_matters.csv')
df2 = pd.read_csv('a_comprehensive_features_data.csv')

In [20]:
df2['ts_code'] = df2['ts_code'].str.split('.').str[0]
df1['tickers'] = df1['tickers'].astype(str).str.zfill(6)
df2['ts_code'] = df2['ts_code'].astype(str).str.zfill(6)
df1.set_index('tickers', inplace=True)
df2.set_index('ts_code', inplace=True)
merged_df = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
print(merged_df.head())

       company listed_market listing_date  issue_price  \
000001    平安银行       深圳证券交易所   1991-04-03         40.0   
000002     万科A       深圳证券交易所   1991-01-29          1.0   
000004    国华网安       深圳证券交易所   1991-01-14          1.0   
000005    ST星源       深圳证券交易所   1990-12-10         10.0   
000006    深振业A       深圳证券交易所   1992-04-27         10.0   

                   company_website  \
000001  http://www.bank.pingan.com   
000002        http://www.vanke.com   
000004      http://www.sz000004.cn   
000005  http://www.fountain.com.cn   
000006       http://www.zhenye.com   

                                      business_address  \
000001  广东省深圳市罗湖区深南东路5047号,中国广东省深圳市福田区益田路5023号平安金融中心B座   
000002       广东省深圳市盐田区大梅沙环梅路33号万科中心,香港中环花园道1号中银大厦43楼A室   
000004                  广东省深圳市福田区梅林街道孖岭社区凯丰路10号翠林大厦12层   
000005                        广东省深圳市罗湖区深南东路2017号华乐大厦3楼   
000006             广东省深圳市南山区科技南路16号深圳湾科技生态园11栋A座42-43层   

                                            main_business  \
000001   人民币

In [21]:
merged_df.to_csv('a_comprehensive_features_final.csv', index=False)